# RLC circuit analysis

A notebook to analyse the data from an investigation from RLC circuits
Rick and Revolio, 22/03/2022

## Import packages

In [ ]:
import pandas as pd # panads for data manipulation
import numpy as np # numpy for all things mathematical/numerical
import matplotlib.pyplot as plt # matplotlib.pyplot for plotting
import glob # glob for file collection
from scipy import optimize # optimize for least squares fitting

Read in the files from the RLC directory, taking only .csv formatted files

In [ ]:
files = glob.glob('RLC/*.csv')

Check that the files open as expected:

In [ ]:
test = files[0]
df = pd.read_csv(test, skiprows = 8, header = None) # We need to skip the first 8 rows as there is a bunch of instrument information that we don't need for this analysis
df

With an individual file working, I want to get the frequency so I can label each data point. Something like this should work:

In [ ]:
freq = int(round(df[df.isin(["Ch A Frequency (Hz)"]).any(axis=1)][1],0))

Now to actually get the data. I wrote many test pieces to get all of this working first, but the general idea is that we read in each file, take the infomation that we need, and the add that to the _data_ dataframe. The only non-trivial things we do: the first time that the loop runs, we take the names of the measured quantities and record these in a list; ultimately we set this to be the index of the dataframe before transposing it it make it useful,

In [ ]:
# Loop over all the files in the RLC directory
for n, f in enumerate(files):
    df = pd.read_csv(f, skiprows = 8, header = None)
    
    # First time around, get the measurement names and store them
    if n == 0:
        headers = df[0]
        data = pd.DataFrame()
        data['Frequencies'] = headers
    
    # Get the frequency from the measured value. This is just a label, it is not actually used in plots or analysis
    freq = int(round(df[df.isin(["Ch A Frequency (Hz)"]).any(axis=1)][1],0))
    data[freq] = df[1] # create a new column in the dataframe, labelled by the frequency above
    
data = data.set_index('Frequencies') # Set the index for the dataframe as the frequencies as extracted in the loop
data = data.T # Transpose the dataframe

With data in a useful format, we can proceed to plotting it. I have done a bit of plotting before, so I am going to use the _grid_spec_ functionality to plot both the phase and amplitude on the same image

In [ ]:
f, A, B, p = data['Ch A Frequency (Hz)'], data['Ch A Amplitude (V)'], data['Ch B Amplitude (V)'], np.pi * data['Ch A - Ch B Phase (deg)']/180 # Values to plot
A_err = A/100
p_err = 0.1

fig = plt.figure(figsize=(12, 10)) # Set the figure size
gs = fig.add_gridspec(nrows=2, ncols=1, height_ratios=[5, 2]) # Set the ratios of sub images
plt.rcParams['font.size'] = 16 # Increase the font size

R, L, C = 1e3, 5e-3, 103e-9
w0 = 1/(2*np.pi*np.sqrt(L * C))
Q = (1/R) * np.sqrt(L/C)
BW = w0/Q

# First image: amplitude response
ax0 = fig.add_subplot(gs[0,:])
ax0.errorbar(f, A, yerr = A_err, fmt='o', color = 'C0', label = 'Voltage')
ax0.errorbar(f, (B-A)/B, yerr = A_err, fmt='x', color = 'C1', label = 'Power')
ax0.axvline(w0, color = 'C2', linestyle = '--', label = '$1/\sqrt{LC}$')
# I plotted this, but I don't think it added much value
# ax0.axvspan(w0-BW/2, w0+BW/2, alpha=0.1, color='C2')
fig.axes[0].set_ylabel('Voltage [V]')  # Set the y-axis label
ax0.set_title('Response in RLC circuit')
fig.axes[0].legend()
plt.xscale('log')


# First image: phase response
ax1 = fig.add_subplot(gs[1,:])
ax1.errorbar(f, p, yerr = p_err, fmt='o', color = 'C3', label = 'Phase')
ax1.set_ylim((-4, 4))
fig.axes[1].set_ylabel('Phase [rad]')
fig.axes[1].set_xlabel('Frequency [Hz]')  # Set the x-axis label
plt.xscale('log')

plt.savefig('RLC.pdf', facecolor='white', transparent=False) # Save the figure
plt.show();